In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

2025-12-05 09:29:53.668765: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')

In [3]:
df.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [4]:
def get_xy(dataframe, y_label='Diabetes_012', X_label=None):
    if X_label:
        X = dataframe[X_label]
    else:    
        X = dataframe.drop('Diabetes_012', axis=1)
    y = dataframe[y_label]
    return X, y

In [5]:
def preprocess_data(dataframe):
    # Separate features and target variable
    X = get_xy(dataframe)[0]
    y = get_xy(dataframe)[1]

    # Standardize the feature variables
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Handle class imbalance using RandomOverSampler
    ros = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = ros.fit_resample(X_scaled, y)

    return X_resampled, y_resampled, scaler

In [6]:
train, test, val = np.split(df.sample(frac=1, random_state=42), [int(.7*len(df)), int(.85*len(df))])
X_train, y_train, scaler = preprocess_data(train)
X_test, y_test, _ = preprocess_data(test)
X_val, y_val, _ = preprocess_data(val)

/Users/jokello/Projects/DS/.venv/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

/Users/jokello/Projects/DS/.venv/lib/python3.12/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=50,
                    batch_size=32)

Epoch 1/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.5562 - loss: 0.9139 - val_accuracy: 0.5101 - val_loss: 1.0165
Epoch 2/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.6053 - loss: 0.8547 - val_accuracy: 0.4906 - val_loss: 1.0935
Epoch 3/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 33s 2ms/step - accuracy: 0.6291 - loss: 0.8202 - val_accuracy: 0.4987 - val_loss: 1.1518
Epoch 4/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.6433 - loss: 0.7984 - val_accuracy: 0.4991 - val_loss: 1.2100
Epoch 5/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.6528 - loss: 0.7825 - val_accuracy: 0.4942 - val_loss: 1.2618
Epoch 6/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 34s 2ms/step - accuracy: 0.6607 - loss: 0.7703 - val_accuracy: 0.4993 - val_loss: 1.3016
Epoch 7/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 40s 3ms/step - accuracy: 0.6655 - loss: 0.7607 - val_accuracy: 0.4920 - val_loss: 1.3531
Epoch 8/50
14025/14025 ━━━━━━━━━━━━━━━━━━━━ 39s 3ms/step - accuracy: 